### 07/31/2024: update_vector and CC function code before properly utilizing mu and handeling rotation issues

In [ ]:
def update_vector(v, theta):
    """Update a vector by rotating it towards the center of the nonnegative orthant."""
    center = np.ones_like(v) / np.sqrt(len(v))
    direction = center - v
    direction /= np.linalg.norm(direction)
    return v + np.sin(theta) * direction

def cone_collapsing_add(U, X, k, theta):
    """
    Cone Collapsing algorithm with alternating vector updates and new vector addition.

    Args:
    U: Initial cone vectors
    X: Data points
    k: Number of new vectors to add (maximum)
    theta: Step size for vector updates
    """
    m, n = X.shape
    iteration = 0
    max_iterations = 1000  # Add a maximum iteration limit
    update_index = 0
    new_vectors_added = 0

    while new_vectors_added < k and iteration < max_iterations:
        iteration += 1

        # Update a vector (rotating through the first 3 vectors)
        update_index = (update_index + 1) % 3
        U[:, update_index] = update_vector(U[:, update_index], theta)

        # Check all points
        for i in range(n):
            x = X[:, i]
            if pseudo_inverse_test(U, x) and combinatorial_test(U, x):
                U = np.column_stack((U, x))
                new_vectors_added += 1
                plot_cone_and_points(U, X, f"Iteration {iteration}: Added vector {new_vectors_added}")
                break  # Only add one vector per iteration

        if new_vectors_added == k:
            break

    return U, iteration

def cone_collapsing_rotate(U, X, theta, initial_rotation_angle, max_rotations):
    """
    Cone Collapsing algorithm with rotating cone approach.

    Args:
    U: Initial cone vectors
    X: Data points
    theta: Step size for vector updates
    rotation_angle: Angle to rotate the cone when a point falls out
    max_rotations: Maximum number of rotations before giving up on a vector update
    """
    m, n = X.shape
    iteration = 0
    max_iterations = 1000
    update_index = 0
    rotation_angle = initial_rotation_angle

    while iteration < max_iterations:
        iteration += 1
        update_index = (update_index + 1) % 3

        # Store original vectors
        U_original = U.copy()

        # Try updating the vector
        U[:, update_index] = update_vector(U[:, update_index], theta)

        # Check if any point falls out
        point_outside = False
        for i in range(n):
            x = X[:, i]
            if pseudo_inverse_test(U, x) and combinatorial_test(U, x):
                point_outside = True
                break
        # point_outside = any(pseudo_inverse_test(U, X[:, i]) and combinatorial_test(U, X[:, i]) for i in range(n)) # for statement above condensed into one line

        if point_outside:
            # Move vectors back one step
            U = U_original

            # Try rotating and updating
            for _ in range(max_rotations):
                # Rotate the entire cone
                U = rotate_cone(U, rotation_angle)

                # Try updating again
                U[:, update_index] = update_vector(U[:, update_index], theta)

                # Check if any point falls out after rotation and update
                point_outside = False
                for i in range(n):
                    x = X[:, i]
                    if pseudo_inverse_test(U, x) and combinatorial_test(U, x):
                        point_outside = True
                        break

                if not point_outside:
                    # Successful update after rotation
                    break

            if point_outside:
                # If still unsuccessful after max_rotations, reduce rotation angle
                rotation_angle /= 2
                U = U_original  # Revert to the state before rotation attempts

        # Ensure all vectors remain nonnegative
        U = np.maximum(U, 0)

        # Plot the current state
        # plot_cone_and_points(U, X, f"Iteration {iteration}")

    return U, iteration

def rotate_cone(U, angle):
    """Rotate the entire cone by the given angle."""
    rotation_matrix = np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]
    ])
    return rotation_matrix @ U